# **Install MMDetection**

In [2]:
!nvcc -V
!gcc --version 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
!pip install -U torch==1.7.1+cu110 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install mmcv-full
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection
%cd mmdetection
!pip install -e .
!pip install Pillow==7.0.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████    | 1009.0 MB 64.2 MB/s eta 0:00:03   |██▎                             | 80.5 MB 57.8 MB/s eta 0:00:19     |██▌                             | 89.5 MB 30.3 MB/s eta 0:00:36     |██▌                             | 92.3 MB 30.3 MB/s eta 0:00:36     |██▊                             | 97.0 MB 30.3 MB/s eta 0:00:36     |████▍                           | 159.5 MB 60.8 MB/s eta 0:00:17MB/s eta 0:00:17     |█████▌                          | 200.6 MB 67.6 MB/s eta 0:00:15     |███████▎                        | 265.0 MB 36.9 MB/s eta 0:00:25     |██████████████▌                 | 522.7 MB 49.8 MB/s eta 0:00:13MB/s eta 0:00:08     |███████████████████             | 690.7 MB 59.5 MB/s eta 0:00:08     |███████████████████▏            | 691.9 MB 59.5 MB/s eta 0:00:08     |███████████████████▊            | 712.9 MB 48.3 MB/s eta 0:00:10     |████████████████████            | 722.1 MB 12.8 MB/s eta 0

# **Import Libraries**

In [10]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

import mmdet as mmdet
print(mmdet.__version__)

from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

import os
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import glob
import cv2
import shutil
import random
import os.path as osp
import json
import numpy as np
import pandas as pd
import mmcv
from mmdet.apis import set_random_seed
import re
import xml.etree.ElementTree as ET
from typing import Dict, List
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

1.7.0 True


ModuleNotFoundError: No module named 'mmdet'

In [11]:
global_seed = 0

def set_seed(seed=global_seed):
    """Sets the random seeds."""
    set_random_seed(seed, deterministic=False)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

NameError: name 'set_random_seed' is not defined

In [12]:
%cd ..

/


# **Convert to COCO Annotations**

In [16]:
with open('../input/oil-storage-tanks/Oil Tanks/labels.json', 'r') as f:
    data = f.read()

FileNotFoundError: [Errno 2] No such file or directory: '/input/oil-storage-tanks/Oil Tanks/labels.json'

In [ ]:
data = json.loads(data)

In [ ]:
data[4]['label']

In [ ]:
annots = []
for annot in data:
    if annot['label'] != 'Skip':
        annots.append(annot['id'])

In [ ]:
%%writefile labels.txt
Tank
Tank Cluster
Floating Head Tank

In [ ]:
non_annots = []
for i in range(len(data)):
    idx = data[i]['id']
    if idx not in annots:
        non_annots.append(idx)

Train, Val Split

In [ ]:
non_annots_sample = random.sample(non_annots, 350)
train_non_annots = random.sample(non_annots_sample, 300)
val_non_annots = []
for idx in non_annots_sample:
    if idx not in train_non_annots:
        val_non_annots.append(idx)

In [ ]:
annots_sample = random.sample(annots, 350)
train_annots = random.sample(annots_sample, 300)
val_annots = []
for idx in annots_sample:
    if idx not in train_annots:
        val_annots.append(idx)

In [ ]:
def convert_to_coco(annot_idx_list, non_annot_idx_list, output_json_path, data=data, labels=['Tank', 'Tank Cluster', 'Floating Head Tank'], starting_bbox_id=1):
    output_json_dict = {
        'images': [],
        'annotations': [],
        'categories' : []
    }
    for annot_idx in annot_idx_list:
        for label in data[annot_idx]['label']:
            if label == labels[0]:
                label_id = 0
            elif label == labels[1]:
                label_id = 1
            elif label == labels[2]:
                label_id = 2
            else:
                raise ValueError('Category is not recognized')
            for rect in data[annot_idx]['label'][label]:
                xmin = rect['geometry'][0]['x']
                ymin = rect['geometry'][0]['y']
                xmax = rect['geometry'][2]['x']
                ymax = rect['geometry'][1]['y']
                bbox_width = xmax - xmin
                bbox_height = ymax - ymin
                bbox_area = bbox_width * bbox_height
                img_id = data[annot_idx]['id']
                bbox_id = starting_bbox_id
                starting_bbox_id = starting_bbox_id + 1
                annot = {
                    'category_id': label_id,
                    'segmentation': [],
                    'area': bbox_area,
                    'bbox': [xmin, ymin, bbox_width, bbox_height],
                    'id': bbox_id,
                    'image_id': img_id,
                    'iscrowd': 0
                }
                output_json_dict['annotations'].append(annot)
        file_name = data[annot_idx]['file_name']
        img_info = {
            'id': img_id,
            'width': 512,
            'height': 512,
            'file_name': file_name
        }
        output_json_dict['images'].append(img_info)
    
    for non_annot_idx in non_annot_idx_list:
        file_name = data[non_annot_idx]['file_name']
        img_id = data[non_annot_idx]['id']
        img_info = {
            'id': img_id,
            'width': 512,
            'height': 512,
            'file_name': file_name
        }
        output_json_dict['images'].append(img_info)
    for i, label in enumerate(labels):
        category_info = {
            'id': i,
            'name': label,
            'supercategory': 'none'
        }
        output_json_dict['categories'].append(category_info)
        
    with open(output_json_path, 'x') as f:
        output_json = json.dumps(output_json_dict)
        f.write(output_json)

In [ ]:
convert_to_coco(annot_idx_list=train_annots, non_annot_idx_list=train_non_annots, output_json_path='/kaggle/working/output.json')

In [ ]:
convert_to_coco(annot_idx_list=val_annots, non_annot_idx_list=val_non_annots, output_json_path='/kaggle/working/val-output.json')

# **Custom Config File With Deformable CNN V2**

In [ ]:
from mmcv import Config
cfg = Config.fromfile('/kaggle/working/mmdetection/configs/dcn/cascade_rcnn_r101_fpn_dconv_c3-c5_1x_coco.py')

In [ ]:
from mmdet.apis import set_random_seed

cfg.dataset_type = 'CocoDataset'
cfg.classes = '/kaggle/working/labels.txt'
cfg.data_root = '/kaggle/working'

for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 3

cfg.data.test.type = 'CocoDataset'
cfg.data.test.classes = 'labels.txt'
cfg.data.test.data_root = '/kaggle/working'
cfg.data.test.ann_file = 'val-output.json'
cfg.data.test.img_prefix = '../input/oil-storage-tanks/Oil Tanks/image_patches'

cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = '/kaggle/working'
cfg.data.train.ann_file = 'output.json'
cfg.data.train.img_prefix = '../input/oil-storage-tanks/Oil Tanks/image_patches'
cfg.data.train.classes = 'labels.txt'

cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = '/kaggle/working'
cfg.data.val.ann_file = 'val-output.json'
cfg.data.val.img_prefix = '../input/oil-storage-tanks/Oil Tanks/image_patches'
cfg.data.val.classes = 'labels.txt'

albu_train_transforms = [
    dict(type='ShiftScaleRotate', shift_limit=0.0625,
         scale_limit=0.15, rotate_limit=15, p=0.4),
    dict(type='RandomBrightnessContrast', brightness_limit=0.2,
         contrast_limit=0.2, p=0.5),
    dict(type='IAAAffine', shear=(-10.0, 10.0), p=0.4),
#     dict(type='MixUp', p=0.2, lambd=0.5),
#     dict(type="Blur", p=1.0, blur_limit=7),
#     dict(type='CLAHE', p=0.5),
#     dict(type='Equalize', mode='cv', p=0.4),
    dict(
        type="OneOf",
        transforms=[
            dict(type="GaussianBlur", p=1.0, blur_limit=7),
            dict(type="MedianBlur", p=1.0, blur_limit=7),
        ],
        p=0.4,
    ),]

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(512, 512), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
        type='BboxParams',
        format='coco',
        label_fields=['gt_labels'],
        min_visibility=0.0,
        filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(512, 512),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

cfg.load_from = '../input/dcn-checkpoint/cascade_rcnn_r101_fpn_dconv_c3-c5_1x_coco_20200203-3b2f0594.pth'

cfg.work_dir = '/kaggle/working/model_output'

cfg.optimizer.lr = 0.02 / 8
cfg.lr_config = dict(
    policy='CosineAnnealing', 
    by_epoch=False,
    warmup='linear', 
    warmup_iters=500, 
    warmup_ratio=0.001,
    min_lr=1e-07)

cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu = 2

cfg.evaluation.metric = 'bbox'
cfg.evaluation.interval = 4

cfg.checkpoint_config.interval = 12
cfg.runner.max_epochs = 12
cfg.log_config.interval = 50

cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

print(f'Config:\n{cfg.pretty_text}')

In [ ]:
datasets = [build_dataset(cfg.data.train)]
model = build_detector(
 cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

In [ ]:
model = init_detector(cfg, '/kaggle/working/model_output/epoch_12.pth')
for i in range(30):
    img = data[annots[500+i]]['file_name']
    img = mmcv.imread('../input/oil-storage-tanks/Oil Tanks/image_patches/' + img)
    result = inference_detector(model, img)
    show_result_pyplot(model, img, result)

In [7]:
# Import necessary libraries
import os
import random
import json
import numpy as np
import mmcv
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, inference_detector, init_detector, show_result_pyplot
from mmdet.apis import set_random_seed
from mmcv import Config

# Set a random seed for reproducibility
set_random_seed(0, deterministic=False)

# Load JSON data containing object annotations
with open('annotations.json', 'r') as f:
    data = json.load(f)

# Separate annotations into annotated and non-annotated categories
annots = [annot['id'] for annot in data if annot['label'] != 'Skip']
non_annots = [annot['id'] for annot in data if annot['label'] == 'Skip']

# Randomly split annotations into training and validation sets
train_annots = random.sample(annots, 300)
val_annots = [idx for idx in annots if idx not in train_annots]

# Randomly sample non-annotations for training and validation sets
non_annots_sample = random.sample(non_annots, 350)
train_non_annots = random.sample(non_annots_sample, 300)
val_non_annots = [idx for idx in non_annots_sample if idx not in train_non_annots]

# Create COCO-style annotation files for training and validation datasets
def convert_to_coco(annot_idx_list, non_annot_idx_list, output_json_path):
    
    # Construct COCO-style annotation dictionary
    output_json_dict = {
        'images': [],
        'annotations': [],
        'categories': []
    }
    
    # Iterate through annotated samples
    for annot_idx in annot_idx_list:
        # Process annotations and add to the output dictionary
    
    # Iterate through non-annotated samples
    for non_annot_idx in non_annot_idx_list:
        # Process non-annotations and add to the output dictionary
    
    # Save the output dictionary as a JSON file
    with open(output_json_path, 'w') as f:
        json.dump(output_json_dict, f)

# Convert annotations to COCO format for training and validation datasets
convert_to_coco(train_annots, train_non_annots, 'train_output.json')
convert_to_coco(val_annots, val_non_annots, 'val_output.json')

# Load a pre-defined MMDetection configuration file
cfg = Config.fromfile('mmdetection_config.py')

# Modify the configuration to match the dataset and training settings
cfg.dataset_type = 'CocoDataset'
cfg.classes = 'labels.txt'
cfg.data_root = 'data/'

# ...

# Train the model
datasets = [build_dataset(cfg.data.train)]
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES
train_detector(model, datasets, cfg, distributed=False, validate=True)

# Initialize the trained model for inference
model = init_detector(cfg, 'model_output/epoch_12.pth')

# Perform inference and visualize detection results on sample images
for i in range(30):
    img = data[annots[500 + i]]['file_name']
    img = mmcv.imread('data/images/' + img)
    result = inference_detector(model, img)
    show_result_pyplot(model, img, result)


IndentationError: expected an indented block (<ipython-input-7-dbc7e4a116d8>, line 47)